In [2]:
import pandas as pd

In [43]:
df = pd.read_hdf("uspto_grant_data_icpr_section_H.h5")

In [57]:
# Build the inverse lookup tables for row indices in the dataframe based on patent and application document ids
applicationLUT = {}
patentLUT = {}

patentApplicationDocNumbers = df["patentApplicationDocNumber"].tolist()
publishedPatentDocNumbers = df["publishedPatentDocNumber"].tolist()
for (idx, docnum) in enumerate(patentApplicationDocNumbers):
    applicationLUT[docnum] = idx
for (idx, docnum) in enumerate(publishedPatentDocNumbers):
    patentLUT[docnum] = idx

In [66]:
# Example of how to use the LUTs: note the document ids are strings
df.ix[applicationLUT["14498577"]]
df.ix[patentLUT["06981300"]]

abstract                      A piezoelectric ink jet print head and a fabri...
applicantCitations                                                           []
applicationDate                                                        20030902
applicationType                                                         utility
examinerCitations                          [5581861, 6096656, 6334670, 6701593]
inventionTitle                Piezoelectric ink jet print head and fabricati...
ipcrType                                                                   H04R
originalFileName              /Users/gittens/Downloads/uspto-grants/ipgb2006...
originalLineNumber                                                          130
patentApplicationDocNumber                                             10653621
publishedPatentDocNumber                                               06981300
Name: 0, dtype: object

In [67]:
# the dataframe
df

,abstract,applicantCitations,applicationDate,applicationType,examinerCitations,inventionTitle,ipcrType,originalFileName,originalLineNumber,patentApplicationDocNumber,publishedPatentDocNumber
0,A piezoelectric ink jet print head and a fabri...,[],20030902,utility,"[5581861, 6096656, 6334670, 6701593]",Piezoelectric ink jet print head and fabricati...,H04R,/Users/gittens/Downloads/uspto-grants/ipgb2006...,130,10653621,06981300
1,A hearing protection device and method of natu...,"[4187838, 4231360, 6056082, D437048, 6302111, ...",20030422,utility,"[3807526, 5835609, 6445805]",Ear clip,H04R,/Users/gittens/Downloads/uspto-grants/ipgb2006...,393,10420172,06981569
2,"A transponder (1) comprising an antenna (4), a...","[4933647, 5699017, 0 722 094, 57192115]",20030613,utility,"[4123718, 4817198, 5331638, 5361395, 5537672, ...",Low power regulated amplifier for a transponder,H04Q,/Users/gittens/Downloads/uspto-grants/ipgb2006...,1447,10460824,06982627
3,A mechanism for assigning an actuator to a dev...,"[4495496, 4837568, 5450087, 285 419, 427 342, ...",19971015,utility,"[4998249, 5357141, 5493283, 5508689, 5515053, ...",Mechanism for assigning an actuator to a device,H04Q,/Users/gittens/Downloads/uspto-grants/ipgb2006...,1448,09297952,06982628
4,A method and device for child seat recognition...,"[195 18 410, 195 19 130, 198 03 682, 11 278210]",20021007,utility,"[3754187, 4852934, 5711574, 5790031, 5793291, ...",Method for child seat recognition in a vehicle,H04Q,/Users/gittens/Downloads/uspto-grants/ipgb2006...,1449,10265841,06982629
5,A vibrating push button station to be associat...,[],20040102,utility,"[4590474, 4851836, 5103223, 5172092, 5339071, ...",Vibrating pedestrian push button station,H04B,/Users/gittens/Downloads/uspto-grants/ipgb2006...,1450,10749848,06982630
6,A system for tracking a subject of interest in...,"[4510495, 4618861, 5150310, 5353011, 5381137, ...",20021126,utility,"[3439320, 4225953, 4275385, 4598275, 5218344, ...",Wireless subject locator,H04Q,/Users/gittens/Downloads/uspto-grants/ipgb2006...,1459,10304568,06982639
7,The invention relates to an internal platform ...,"[6292750, 00 48014]",20020924,utility,"[5654890, 6161062, 6424914, 6785609]",Hybrid inertial navigation system with improve...,H04B,/Users/gittens/Downloads/uspto-grants/ipgb2006...,1489,10488941,06982669
8,A reduced area imaging device is provided whic...,"[4491865, 4745471, 4786965, 4814648, 4854302, ...",20011004,utility,"[6141037, 6232589, 6243131, 6417882]",Reduced area imaging devices utilizing selecte...,H04N,/Users/gittens/Downloads/uspto-grants/ipgb2006...,1560,09971749,06982740
9,The method for coding live images in microscop...,[5216596],20010523,utility,"[4673988, 5548661, 6101265, 6226392, 6259473, ...",Method and apparatus for coding live images in...,H04N,/Users/gittens/Downloads/uspto-grants/ipgb2006...,1561,09863701,06982741


In [56]:
df.ix[187835]

abstract                      The present invention provides a new system st...
applicantCitations                                                           []
applicationDate                                                        20140122
applicationType                                                         utility
examinerCitations                                                            []
inventionTitle                Mobile cellular network based on layered cloud...
ipcrType                                                                   H04W
originalFileName              /Users/gittens/Downloads/uspto-grants/ipgb2016...
originalLineNumber                                                         6700
patentApplicationDocNumber                                             14161518
publishedPatentDocNumber                                               09439243
Name: 187835, dtype: object

abstract                      Provided is an apparatus and method for perfor...
applicantCitations                                                           []
applicationDate                                                        20140926
applicationType                                                         utility
examinerCitations                                                            []
inventionTitle                Method and apparatus for performing activation...
ipcrType                                                                   H04W
originalFileName              /Users/gittens/Downloads/uspto-grants/ipgb2016...
originalLineNumber                                                         6690
patentApplicationDocNumber                                             14498577
publishedPatentDocNumber                                               09439233
Name: 187825, dtype: object